## Traduction
Accès au dossier du projet sur Google Drive

In [ ]:
import sys
import os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive/', force_remount=True)
    %cd drive/MyDrive/nov23_bds_rakuten
else:
    root_files = os.listdir()
    if not "requirements.txt" in root_files:
        print("The jupyter server root should be set to root of git project")
        print("If you're on vscode add the following to your settings.json file")
        print('"jupyter.notebookFileRoot": "${workspaceFolder}"')

# Importation des librairies

In [ ]:
!pip install googletrans==4.0.0-rc1


In [ ]:
import pandas as pd
from googletrans import Translator, LANGUAGES
import json
import os
from requests.exceptions import ReadTimeout

Chargement du fichier source \data\output\preprocessing-lang\X_train_pre.csv

In [ ]:
df = pd.read_csv('output/data-exploration/X_train_pre.csv', index_col=0)
print(df.head())


# Fonction pour traduire le texte en français

La fonction envoi une requête à google traduction. Elle doit pouvoir segmenter le texte pour respecter la limite de 5000 caractères.

Elle doit aussi pouvoir être relancé en cas d'erreur car le traitement peut être très long et le service google peut arrêter de répondre si on dépasse une certaine limite d'utilisation.

Attention : fonction récursive sur ReadTimeOut!!!

In [ ]:
# Initialiser le traducteur
translator = Translator()


# Fonction pour traduire le texte
def translate_text(df, start_index = 0, save_path = "output/traduction_fr/translation.json"):
    """ 
    Traduit la colonne texte d'un DataFrame vers le français et sauvegarde le résultat dans save_path au fur et à mesure (gestion des refus de google translate)
    df : le DataFrame traduit. Doit contenir une colonne 'text' à traduire et 'lang', la langue d'origine (fr, en, de)
    start_index : permet de reprendre après une erreur au dernier index traité 
    save_path : chemin de sauvegarde du fichier de traduction

    """
    translations = {}
    # Charger les traductions existantes si le fichier existe
    if os.path.exists(save_path):
        with open(save_path, "r") as file:
            translations = json.load(file)

    if start_index > len(df):
        print("Job terminé : Aucune ligne restante à traduire.")
        return
        
    for i, row in df.iloc[start_index:].iterrows():
        try:
            if row['lang'] in LANGUAGES and row['lang'] != 'fr':       # Si le code langue est reconnu et différent de en
                # Vérifier la longueur du texte et le couper si nécessaire
                texte = row['text']
                src_lang = row['lang']
                if len(texte) > 4000:
                    # Couper le texte en morceaux de 4000 caractères
                    parts = [texte[i:i+4000] for i in range(0, len(texte), 4000)]
                    translated_text = ' '.join([translator.translate(part, src=src_lang, dest='fr').text for part in parts]) #Traduire en fr
                else:
                    translated_text = translator.translate(texte, src = src_lang, dest = 'fr').text     #Traduire en fr
            else:      
                translated_text = row['text']  # sinon, ajouter la ligne en l'état
            translations[i] = translated_text
            # Sauvegarde intermédiaire
            with open(save_path, "w") as file:
                json.dump(translations, file)
        except ReadTimeout:
            # En cas d'erreur de timeout, recommencer depuis la dernière position sauvegardée
            print(f"ReadTimeout à l'index {i}. Tentative de redémarrage.")
            translate_text(df, start_index = i, save_path = save_path)
            break
        except AttributeError as e:
            # Enregistrer l'index d'erreur et arrêter
            print(f"Erreur à l'index {i}: {e}")
            break


# Test fonctionnels sur la fonction

In [ ]:
test = pd.DataFrame({
  "index": [1, 2, 3],
  "texte": ["Bonjour", "Je teste la fonction", "de traduction automatique"],
  "lang_texte": ["fr", "fr", "fr"]
})


# En cas de reprise après erreur
# Vérifie s'il existe un fichier de sauvegarde et reprise à partir de l'index le plus élevé
try:
    translations_dict = pd.read_json("transla_test.json", orient='index')
    translations_df = translations_dict.rename(columns={0: 'texte_en'})
    translate_text(test, start_index = translations_df.value_counts().sum(), save_path = 'transla_test.json')
except (FileNotFoundError, ValueError):
    # S'il n'y a pas de fichier ou s'il est vide, commencer depuis le début
    translate_text(test, save_path = 'transla_test.json')


In [ ]:
translations_dict = pd.read_json("transla_test.json", orient='index')
translations_df = translations_dict.rename(columns={0: 'texte_en'})

print(translations_df.tail(10))
translations_df.value_counts().sum()

# Traduire le dataframe df (X_train_pre.csv)

Environ 750 lignes / 10 minutes !!!
Il y a plus de 84000 lignes -> environ 20h

In [ ]:
# En cas de reprise après erreur
# Vérifie s'il existe un fichier de sauvegarde et reprise à partir de l'index le plus élevé
try:
    translations_dict = pd.read_json("output/traduction_fr/translation.json", orient='index')
    translations_df = translations_dict.rename(columns={0: 'text_fr'})
    translate_text(df, start_index = translations_df.value_counts().sum(), save_path = 'output/traduction_fr/translation.json')
except (FileNotFoundError, ValueError):
    # S'il n'y a pas de fichier ou s'il est vide, commencer depuis le début
    translate_text(df, save_path = 'output/traduction_fr/translation.json')

Charge le fichier généré. 
Affiche les dernières valeurs de translation.json et le nombre de lignes

In [ ]:
translations_dict = pd.read_json("output/traduction_fr/translation.json", orient='index')
translations_df = translations_dict.rename(columns={0: 'texte_fr'})

print(translations_df.tail(10))
translations_df.value_counts().sum()

In [ ]:
df.iloc[1034,:]['text']

# Traduire le dataframe X (X_preprocess_to_trad.csv)

Environ  lignes / 10 minutes 
Il y a plus de 84000 lignes -> environ 

In [ ]:
# En cas de reprise après erreur
# Vérifie s'il existe un fichier de sauvegarde et reprise à partir de l'index le plus élevé
try:
    translations_dict = pd.read_json("output/data-exploration/translation_X_preprocess_to_trad.json", orient='index')
    translations_X = translations_dict.rename(columns={0: 'texte_en'})
    translate_text(X, start_index = translations_X.value_counts().sum(), save_path = 'output/data-exploration/translation_X_preprocess_to_trad.json')
except (FileNotFoundError, ValueError):
    # S'il n'y a pas de fichier ou s'il est vide, commencer depuis le début
    translate_text(X, save_path = 'output/data-exploration/translation_X_preprocess_to_trad.json')

In [ ]:
translations_dict = pd.read_json("output/data-exploration/translation_X_preprocess_to_trad.json", orient='index')
translations_df = translations_dict.rename(columns={0: 'texte_en'})

print(translations_df.tail(10))
translations_df.value_counts().sum()